In [1]:
import pandas as pd
import numpy as np
import json
import ast
import re
import os
import sys
sys.path.append(os.getcwd()+'../')

In [2]:
# json切り分け後のcsvを取得
# 切り分け後のcsvファイルの場所を指定してください
train = pd.read_csv('../data/input/train.csv')
test = pd.read_csv('../data/input/test.csv')

# Json切り分けされたカラム名と、切り分け不要なカラム名をそれぞれ配列に追加
CATEGORY_NAMES = ['original_language', 'status']
JSON_CATEGORY_NAMES = ['genres_name', 'genres_id']

# 正規表現で使用されている文字
SEPARATE_CHAR = ','

In [3]:
def get_columns_json(train, test, col_name):
    """カラムごとに重複なしのカテゴリの配列を作成する
    
    train : 教師データ
    test : テストデータ
    col_name : カテゴリ分けの対象カラム
    戻り値 : カテゴリ名が格納された配列
    """
    
    data_list = pd.concat([train[col_name], test[col_name]], axis=0)
    set_list = set()
    
    for data in data_list:
        
        if pd.isnull(data):
            continue
        
        tmp_list = re.split(SEPARATE_CHAR, str(data))
        
        for tmp in tmp_list:
            set_list.add(tmp)
        
    return list(set_list)


def check_and_flg(data_vector, key):
    """
    data_vector(pd.DataFrame1列)の文字列の中にkeyが含まれているか
    1行ごとにチェックする
    含まれていれば、含まれているインデックスと同じインデックスの
    列ベクトルの値を 1 にする。含まれていなければ 0 にする
    
    data_vector : pandas.DataFrameの列ベクトル
    key : 検索したい文字列
    戻り値 : フラグ立てされたベクトル(numpy.ndarray)
    """
    
    flags = np.zeros(len(data_vector))
    
    for i, data in enumerate(data_vector):
        if pd.isnull(data):
            flags[i] = int(0)
            continue
        
        flags[i] = int(np.where(re.search(key, str(data))==None, 0, 1))
        
    return flags


def category_to_dummy_json(train, test, col_list=JSON_CATEGORY_NAMES):
    """json切り分けカラムをダミー変数に変換
    
    引数に教師データとテストデータ両方が必要な理由
    => 教師データには存在するがテストデータには存在しないジャンルなどに対応するため
    
    train : json切り分け後の教師データ
    test : json切り分け後のテストデータ
    戻り値 : tr_d, te_d ともにjson切り分けカラムについてのダミー変数
    """
    
    # 空のデータフレームを作成する
    # tr_d : 教師データのダミー変数
    # te_d : テストデータのダミー変数
    tr_d = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    te_d = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    
    # json切り分けを行なったカラムごとに回す
    for col in col_list:
        category_list = get_columns_json(train, test, col)
        
        for i, category_name in enumerate(category_list):
            # カラム名の作成
            col_name = col + '_' + category_name
            
            # カテゴリごとのダミー変数のベクトルを作成
            tmp_tr = pd.DataFrame({col_name:list(check_and_flg(train[col],category_name))})
            tmp_te = pd.DataFrame({col_name:list(check_and_flg(test[col],category_name))})
            
            # ダミー変数のベクトルをつなぎ合わせていく
            tr_d = tmp_tr if len(tr_d)==0 else pd.concat([tr_d, tmp_tr], axis=1)
            te_d = tmp_te if len(te_d)==0 else pd.concat([te_d, tmp_te], axis=1)
    
    return tr_d, te_d


def category_to_dummy(data):
    """Json切り分け不要なカラムをダミー変数に変換
    
    data : pandas.DataFrame型の教師データかテストデータ
    """
    
    dummies = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)
    
    for cat_name in CATEGORY_NAMES:
        tmp = pd.get_dummies(train[cat_name], prefix=cat_name)
        
        # dummiesの length=0(DataFrameが空) であれば dummies=tmp
        # length>0 ならばdummiesの列にtmpを結合する
        dummies = tmp if len(dummies) == 0 else pd.concat([dummies, tmp], axis=1)
    
    return dummies